# Indexing, Selection, and Operations

Maintenant que nous connaissons bien les structures de données des pandas, nous pouvons nous intéresser à certaines des caractéristiques intermédiaires des cadres de données, notamment
    
- le découpage, l'indexation et les sous-ensembles de grands ensembles de données sur la base d'une étiquette intelligente
- **L'étiquetage hiérarchique** des axes
- **Le tri et le classement** des données dans les DataFrames
- La manipulation aisée des **données manquantes**.
- Des outils de **summarisation** des données


Dans cette section, nous allons manipuler les données recueillies auprès des navires de haute mer sur la côte est des Etats Unis. Les opérations des navires sont surveillées à l'aide du **Automatic Identification System (AIS)**, une technologie de navigation de sécurité en mer que les navires sont tenus d'entretenir et qui utilise des transpondeurs pour transmettre des signaux radio à très haute fréquence (VHF) contenant des informations statiques, notamment le nom du navire, l'indicatif d'appel et le pays d'origine, ainsi que des informations dynamiques propres à un voyage particulier, telles que la position, le cap et la vitesse du navire. 

![AIS](images/ais.gif)

La Convention internationale pour la sauvegarde de la vie humaine en mer de l'Organisation Maritime Internationale (OMI) exige des capacités AIS fonctionnelles sur tous les navires de plus de 300 tonnes et les garde-côtes américains exigent l'AIS sur presque tous les navires naviguant dans les eaux américaines. Les gardes-côtes ont mis en place un réseau national de récepteurs AIS qui couvre la quasi-totalité des eaux américaines. **Les signaux AIS** sont transmis plusieurs fois par minute et le réseau est capable de traiter des milliers de rapports par minute et des mises à jour aussi souvent que toutes les deux secondes. Par conséquent, un voyage typique dans notre étude peut inclure la transmission de centaines ou de milliers de signaux codés AIS. Cela fournit une riche source de données spatiales qui comprend des informations à la fois **spatiales et temporelles**.

Pour nos besoins, nous utiliserons des **données résumées** qui décrivent le transit d'un navire donné dans une zone administrative particulière. Les données comprennent l'heure de début et de fin du segment de transit, ainsi que des informations sur la vitesse du navire, la distance parcourue, etc.

In [ ]:
import pandas as pd
import numpy as np

vessels = pd.read_csv('./data/AIS/vessel_information.csv', index_col=0)

## Indexing et Selection

L'indexation est similaire à celle qu'on utilise dans des arrays, à l'exception près qu'on peut extraire les objets `Index`, en plus des index entiers.

In [87]:
# Exemple de Series index
flag = vessels.flag
flag

mmsi
1            Unknown
9            Unknown
21           Unknown
74           Unknown
103          Unknown
              ...   
919191919    Unknown
967191190    Unknown
975318642    Unknown
987654321    Unknown
999999999    Unknown
Name: flag, Length: 10771, dtype: object

In [88]:
# Numpy-style indexing
flag[:10]

mmsi
1                      Unknown
9                      Unknown
21                     Unknown
74                     Unknown
103                    Unknown
310                   Bermuda 
3011                 Anguilla 
4731      Yemen (Republic of) 
15151                  Unknown
46809    Syrian Arab Republic 
Name: flag, dtype: object

In [89]:
# Indexing par label
flag[[298716,725011300]]

mmsi
298716       Unknown
725011300     Chile 
Name: flag, dtype: object

Dans un `DataFrame` nous pouvons faire des sélections sur les deux axes :

In [90]:
vessels[['num_names','num_types']].head()

,num_names,num_types
mmsi,,
1,8,4
9,3,2
21,1,1
74,2,1
103,3,2


In [91]:
vessels[vessels.max_loa>700]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


L'attribut d'index `loc` permet de choisir des sous-ensembles de manière intuitive :

In [92]:
vessels.loc[720768000, ['names','flag', 'type']]

names             Rusty Pelican
flag     Bolivia (Republic of) 
type                     Towing
Name: 720768000, dtype: object

In [93]:
vessels.loc[:4731, 'names']

mmsi
1       Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...
9                              000000009/Raven/Shearwater
21                                          Us Gov Vessel
74                                      Mcfaul/Sarah Bell
103              Ron G/Us Navy Warship 103/Us Warship 103
310                                              Arabella
3011                                           Charleston
4731                                            000004731
Name: names, dtype: object

La sélection peut se faire aussi avec des labels texte:

In [94]:
vessels.loc[:310, 'flag':'loa']

,flag,flag_type,num_loas,loa
mmsi,,,,
1,Unknown,Unknown,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0
9,Unknown,Unknown,2,50.0/62.0
21,Unknown,Unknown,1,208.0
74,Unknown,Unknown,1,155.0
103,Unknown,Unknown,2,26.0/155.0
310,Bermuda,Foreign,1,47.0


En plus de l'attribut `loc` qui permet de faire une séléction par **label**, pandas permet aussi la séléction par **position** avec l'attribut `iloc`.

Ainsi, nous pouvons demander une position absolue dans le tableau : 

In [95]:
vessels.iloc[:5, 5:8]

,num_loas,loa,max_loa
mmsi,,,
1,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0,156.0
9,2,50.0/62.0,62.0
21,1,208.0,208.0
74,1,155.0,155.0
103,2,26.0/155.0,155.0


### Exercice

Vous pouvez utiliser la méthode `isin` pour faire une requête dans le DataFrame selon une liste de valeurs : 

    data['color'].isin(['red', 'blue'])

Utiliser `isin` pour trouver tous les navires enregistrés au Danemark ou au Japon. Combien d'entrées nous trovons ?

In [ ]:
# Write your answer here

## Indexing avec `where`

Les `DataFrame` Pandas  ont aussi un appel de type `where` qui accepte des conditions. Les objets retournés gardent l'index du `DataFrame` original, qui ne change pas de forme. Ceci est très important pour préserver l' **allignment** lors des opérations entre `DataFrame`s.

In [96]:
np.random.seed(42)
normal_vals = pd.DataFrame({'x{}'.format(i):np.random.randn(100) for i in range(5)})

normal_vals.head()

,x0,x1,x2,x3,x4
0,0.496714,-1.415371,0.357787,-0.828995,-1.594428
1,-0.138264,-0.420645,0.560785,-0.560181,-0.599375
2,0.647689,-0.342715,1.083051,0.747294,0.005244
3,1.523030,-0.802277,1.053802,0.610370,0.046981
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


In [97]:
normal_vals.where(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,0.496714,NaN,0.357787,NaN,NaN
1,NaN,NaN,0.560785,NaN,NaN
2,0.647689,NaN,1.083051,0.747294,0.005244
3,1.523030,NaN,1.053802,0.610370,0.046981
4,NaN,NaN,NaN,NaN,NaN


`where` inclut une option `other` qui accepte une valeur scalaire ou tabulaire pour remplacer la valeur de `DataFrame` si la condition n'est pas satisfaite.

Par exemple, nous pouvons retourner la valeur absolue de X comme ici :

In [98]:
normal_vals.where(normal_vals>0, other=-normal_vals).head()

,x0,x1,x2,x3,x4
0,0.496714,1.415371,0.357787,0.828995,1.594428
1,0.138264,0.420645,0.560785,0.560181,0.599375
2,0.647689,0.342715,1.083051,0.747294,0.005244
3,1.523030,0.802277,1.053802,0.610370,0.046981
4,0.234153,0.161286,1.377669,0.020902,0.450065


Il est aussi possible de faire appel à une fonction `lambda` afin de modifier la valeur :

In [99]:
normal_vals.where(normal_vals>0, other=lambda y: -y*100).head()

,x0,x1,x2,x3,x4
0,0.496714,141.537074,0.357787,82.899501,159.442766
1,13.826430,42.064532,0.560785,56.018104,59.937502
2,0.647689,34.271452,1.083051,0.747294,0.005244
3,1.523030,80.227727,1.053802,0.610370,0.046981
4,23.415337,16.128571,137.766937,2.090159,45.006547


L'opération `mask` est l'inverse booléenne de `where` :

In [100]:
normal_vals.mask(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,NaN,-1.415371,NaN,-0.828995,-1.594428
1,-0.138264,-0.420645,NaN,-0.560181,-0.599375
2,NaN,-0.342715,NaN,NaN,NaN
3,NaN,-0.802277,NaN,NaN,NaN
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


## Sélection avec  `query`

Parfois, la sélection avec les index peut demander une commande trop élaborée car il faut associer plusieurs appels au Dataframe.

In [101]:
normal_vals[(normal_vals.x1 > normal_vals.x0) & (normal_vals.x3 > normal_vals.x2)].head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


Pour une syntaxe plus lisible et concise, nous pouvons utiliser la méthode `query`. Au lieu d'entrer toute la spécification de la colonne, nous pouvons simplement passer une string. Voici le même exemple réécrit avec query :

In [102]:
normal_vals.query('(x1 > x0) & (x3 > x2)').head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


Les objets `DataFrame.index` et `DataFrame.columns` sont inclus par défaut dans les requêtes query. Is nous souhaitons passer une variable, nous pouvons utiliser le préfixe `@`:

In [103]:
min_loa = 700

In [104]:
vessels.query('max_loa > @min_loa')

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


## Opérations

Les objets `DataFrame` et `Series` supportent plusieurs opérations au sein d'un objet ou entre différents objets.

Dans cette section nous allons charger un dataset supplémentaire avec des informations sur le déplacements de chaque navire. Ce dataset est bien plus important car chaque navire a fait plusieurs voyages au long des années.

Par exemple, nous pouvons faire des opérations arithmetiquessur des éléments des deux objets, en combinant par exemple les distances voyagées au long des années. Commençons d'abord à créer des Series pour regrouper les distances parcouroues en 2009 et 2010 :

In [105]:
segments = pd.read_csv('./data/AIS/transit_segments.csv', parse_dates=['st_time'])

Ensuite, on extrait l'année (on verra plus sur les données temporelles d'ici peu)

In [106]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

0    2009
1    2009
2    2009
3    2009
4    2009
Name: year, dtype: int64

In [107]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum() 
                         for name in segments2009.name.unique()})

In [108]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum() 
                         for name in segments2010.name.unique()})

Autre option plus efficace est d'utiliser groupby, qu'on verra plus tard.

In [109]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Maintenant, on peut faire la somme des distances parcourues :

In [110]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

$ Vega Azurit $       NaN
+ Alida Gorthon +     NaN
+alida Gorthon+      63.0
+margit Gorthon+      NaN
0mega Theodore        NaN
                     ... 
Zouzou                NaN
Zuma                  NaN
\m/- /+               NaN
stadt Gera            NaN
vega Saturn           NaN
Length: 7210, dtype: float64

L'alignement des données de Pandas inclut des valeurs `NaN` pour les labels qui ne sont pas communs aux deux Series. En effet, on voit que la plupart des navires n'a que des données pour une seule année.

In [113]:
seg_lengths.notnull().mean()

0.35575589459084606

Bien que cela correspond à la réalité, on ne veut pas de valeurs `NaN` dans notre tableau. nous pouvons remplacer les valeurs `NaN` lors de la somme en indiquant d'utiliser zéro à la place d'un `NaN` :

In [114]:
lengths2009.add(lengths2010, fill_value=0)

$ Vega Azurit $       69.0
+ Alida Gorthon +    497.2
+alida Gorthon+       63.0
+margit Gorthon+     342.3
0mega Theodore        62.6
                     ...  
Zouzou               188.5
Zuma                  54.2
\m/- /+               17.2
stadt Gera            41.8
vega Saturn          161.6
Length: 7210, dtype: float64

Les opérations peuvent aussi être diffusées (**broadcast**) entre les lignes et colonnes.

Par exemple, si on soustrait la valeur maximum de LOA (longueur du navire) de la colonne `max_loa` column, nous avons la différence de taille entre les navires et le plus long des bateaux (en pieds, bien sûr).

In [116]:
vessels.max_loa - vessels.max_loa.max()

mmsi
1           -656.0
9           -750.0
21          -604.0
74          -657.0
103         -657.0
             ...  
919191919   -792.0
967191190   -781.0
975318642   -792.0
987654321   -789.0
999999999   -782.0
Name: max_loa, Length: 10771, dtype: float64

Nous pouvons aussi comparer des élements  ligne par ligne :

In [117]:
vessels[vessels.max_loa==vessels.max_loa.max()]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo


In [118]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

,num_names,num_loas,max_loa,num_types
mmsi,,,,
1,7.0,4.0,-656.0,3.0
9,2.0,-1.0,-750.0,1.0
21,0.0,-2.0,-604.0,0.0
74,1.0,-2.0,-657.0,0.0
103,2.0,-1.0,-657.0,1.0
310,0.0,-2.0,-765.0,0.0
3011,0.0,-2.0,-652.0,0.0
4731,0.0,-2.0,-782.0,0.0
15151,1.0,-1.0,-637.0,0.0


Finalement, nous pouvons aussi appliquer des fonctions aux lignes ou colonnes d'un `DataFrame`

In [119]:
recs.apply(np.median)

num_names      1.0
num_loas       1.0
max_loa      183.0
num_types      1.0
dtype: float64

In [120]:
def range_calc(x):
    return x.max() - x.min()

In [121]:
recs.apply(range_calc)

num_names    167.0
num_loas      83.0
max_loa      812.0
num_types     13.0
dtype: float64

## Tri et Classement

Les objets Pandas incluent des méthodes pour réordonner les données.

In [122]:
segments.sort_index().head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,1,Us Govt Ves,1,1,5.1,13.2,9.2,14.5,96.5,2009-02-10 16:03:00,2/10/09 16:27,2009
1,1,Dredge Capt Frank,1,1,13.5,18.6,10.4,20.6,100.0,2009-04-06 14:31:00,4/6/09 15:20,2009
2,1,Us Gov Vessel,1,1,4.3,16.2,10.3,20.5,100.0,2009-04-06 14:36:00,4/6/09 14:55,2009
3,1,Us Gov Vessel,2,1,9.2,15.4,14.5,16.1,100.0,2009-04-10 17:58:00,4/10/09 18:34,2009
4,1,Dredge Capt Frank,2,1,9.2,15.4,14.6,16.2,100.0,2009-04-10 17:59:00,4/10/09 18:35,2009


In [123]:
segments.sort_index(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
262525,999999999,Triple Attraction,8,1,19.8,18.6,16.1,19.5,100.0,2010-06-18 10:19:00,6/18/10 11:22,2010
262524,999999999,Triple Attraction,7,1,31.5,14.2,13.4,15.1,100.0,2010-06-18 02:52:00,6/18/10 5:03,2010
262523,999999999,Triple Attraction,6,1,17.4,17.0,14.7,18.4,100.0,2010-06-17 19:16:00,6/17/10 20:17,2010
262522,999999999,Triple Attraction,4,1,18.7,19.2,18.4,19.9,100.0,2010-06-15 21:32:00,6/15/10 22:29,2010
262521,999999999,Triple Attraction,3,1,5.3,20.0,19.6,20.4,100.0,2010-06-15 12:49:00,6/15/10 13:05,2010


Essayez d'ordonner les **colonnes** en ordre croissant, au lieu des lignes :

In [124]:
segments.sort_index(axis=1).head()

,avg_sog,end_time,max_sog,min_sog,mmsi,name,pdgt10,seg_length,segment,st_time,transit,year
0,13.2,2/10/09 16:27,14.5,9.2,1,Us Govt Ves,96.5,5.1,1,2009-02-10 16:03:00,1,2009
1,18.6,4/6/09 15:20,20.6,10.4,1,Dredge Capt Frank,100.0,13.5,1,2009-04-06 14:31:00,1,2009
2,16.2,4/6/09 14:55,20.5,10.3,1,Us Gov Vessel,100.0,4.3,1,2009-04-06 14:36:00,1,2009
3,15.4,4/10/09 18:34,16.1,14.5,1,Us Gov Vessel,100.0,9.2,1,2009-04-10 17:58:00,2,2009
4,15.4,4/10/09 18:35,16.2,14.6,1,Dredge Capt Frank,100.0,9.2,1,2009-04-10 17:59:00,2,2009


Il est aussi possible d'utiliser `sort_values` pour trier un `Series`par valeur au lieu du label.

In [125]:
segments.seg_length.sort_values(ascending=False).head(10)

181047    1882.0
181108    1691.0
149218    1675.3
49709     1647.8
149236    1644.2
50604     1557.0
106374    1517.3
140513    1502.4
140555    1491.6
50665     1489.2
Name: seg_length, dtype: float64

Pour un `DataFrame`, nous pouvons trier selon les valeurs de plusieurs colonnes en passant un argument `by` à  `sort_values` :

In [126]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True], 
                                           by=['max_sog', 'min_sog']).head(10)

,avg_sog,max_sog,min_sog
82222,1.5,49.9,0.0
167724,22.3,49.9,0.0
215785,2.8,49.9,0.0
119381,6.6,49.9,0.1
12479,12.2,49.9,3.5
119476,1.9,49.8,0.0
160406,9.9,49.8,0.0
162241,1.9,49.8,0.0
146139,9.6,49.8,0.1
160395,10.1,49.8,0.1


## Hierarchical indexing

While the `mmsi` (Maritime Mobile Service Identity) field represents a unique identifier in the `vessels` table, it does not in the `segments` table.

We can use hierarchical indexing to build a **unique index** that may be more meaningful than the ordered set of integers that are given as a default index.

In [ ]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head(10)

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [ ]:
segments_hier.index.is_unique

Try using this hierarchical index to retrieve the first segment of the tenth transit of the *Sentinel* (mmsi=366766980):

In [ ]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [ ]:
segments.loc[9]

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

If you want to get fancy, both the row and column indices themselves can be given names:

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

With this, we can do all sorts of custom indexing:

In [ ]:
frame.loc['a', 'Ohio']

#### Exercise: Try retrieving the value corresponding to `b2` in `Colorado`:

In [ ]:
# Write your answer here

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [ ]:
segments_hier.swaplevel('mmsi', 'name').head()

In [ ]:
segments_hier.swaplevel('mmsi', 'name').loc['Sentinel'].head()

Alternately, one can index based on an arbitrary level using the `xs` (cross-section) method:

In [ ]:
segments_hier.xs('Sentinel', level='name').head()

Perhaps the easiest way to extract data from arbitrary levels of a `MultiIndex` is to use the `IndexSlice` function. For example, if we want the first two transits from all trips in 2009 and 2010:

In [ ]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

The `:` tells pandas to include the entire level. What `IndexSlice` does is generate a Python `slice` object that represents what is required from each level. So, the above is equivalent to:

    (slice(None, None, None), slice(None, None, None), [2009, 2010], [1, 2])

Data can also be sorted by any index level, using `sort_index` with the `level` argument appropriately specified:

In [ ]:
segments.sort_index(level='max_sog', ascending=False).head()

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

In [ ]:
foo.isnull()

To illustrate how to deal with missing values in pandas, we will import an educational database, which reports test scores for children with hearing disabilities, along with associated covariates, several of which are missing.

In [ ]:
testing = pd.read_csv('./data/test_scores.csv', index_col=0)
testing.head(10)

Missing values may be dropped or indexed out:

By default, `dropna` drops entire rows in which one or more values are missing.

In [ ]:
testing.dropna().head(10)

A similar result can be obtained via indexing.

In [ ]:
testing.isnull().head()

In [ ]:
testing[testing.notnull()].head()

This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [ ]:
testing.dropna(how='all')

This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [ ]:
testing.dropna(thresh=10)

This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [ ]:
# Write your answer here

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [ ]:
testing.fillna(0).head(10)

In [ ]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

Missing values can also be interpolated, using any one of a variety of methods:

In [ ]:
testing.fillna(method='bfill').head(10)

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [ ]:
testing.sum()

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [ ]:
testing.mean()

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
testing.mean(skipna=False)

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [ ]:
testing.describe()

`describe` can detect non-numeric data and sometimes yield useful information about it.

In [ ]:
testing.non_english.describe()

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
testing.score.cov(testing.age_test)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
testing.score.corr(testing.age_test)

Try running `corr` on the entire `testing` DataFrame to see what is returned:

In [ ]:
# Write answer here

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
segments_hier.head()

In [ ]:
segments_hier.sum(level='transit').seg_length

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [ ]:
testing.to_csv("testing.csv")

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

An efficient way of storing data to disk is in binary format. Pandas supports this using Python’s built-in pickle serialization.

In [ ]:
testing.to_pickle("testing_pickle")

The complement to `to_pickle` is the `read_pickle` function, which restores the pickle to a `DataFrame` or `Series`:

In [ ]:
pd.read_pickle("testing_pickle").head()

As Wes warns in his book, it is recommended that binary storage of data via pickle only be used as a temporary storage format, in situations where speed is relevant. This is because there is no guarantee that the pickle format will not change with future versions of Python.

## References

[Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney